In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import glob

import cv2

import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


2022-09-22 16:45:25.885333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:25.914851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:25.914987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
ds_train = image_dataset_from_directory(
'tiny_imagenet/train',
image_size=(64,64),
batch_size=32)

Found 100000 files belonging to 200 classes.


2022-09-22 16:45:31.940395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:31.940548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:31.940655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:32.461095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-22 16:45:32.461235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [4]:
ds_val = image_dataset_from_directory(
'tiny_imagenet/val',
image_size=(64,64),
batch_size=32)

Found 10000 files belonging to 200 classes.


In [189]:
# conv_base = keras.applications.vgg19.VGG19(
# weights="imagenet",
# input_shape=(64, 64, 3),
# include_top=False)

# conv_base.trainable = False
# for layer in conv_base.layers:
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [5]:
inputs = keras.Input(shape=(64,64,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [6]:
opt = tf.optimizers.Adam(learning_rate=0.0001)

In [7]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [8]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_without_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [9]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40


2022-09-22 16:45:51.919332: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8303
2022-09-22 16:45:53.149437: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3125/3125 [==============================] - 84s 26ms/step - loss: 5.0058 - accuracy: 0.0261 - val_loss: 4.7524 - val_accuracy: 0.0425
Epoch 2/40
3125/3125 [==============================] - 81s 26ms/step - loss: 4.5022 - accuracy: 0.0636 - val_loss: 4.2078 - val_accuracy: 0.1038
Epoch 3/40
3125/3125 [==============================] - 81s 26ms/step - loss: 4.2015 - accuracy: 0.0960 - val_loss: 4.1600 - val_accuracy: 0.1108
Epoch 4/40
3125/3125 [==============================] - 80s 26ms/step - loss: 3.9872 - accuracy: 0.1239 - val_loss: 3.7770 - val_accuracy: 0.1630
Epoch 5/40
3125/3125 [==============================] - 80s 26ms/step - loss: 3.8171 - accuracy: 0.1502 - val_loss: 3.7121 - val_accuracy: 0.1756
Epoch 6/40
3125/3125 [==============================] - 81s 26ms/step - loss: 3.6807 - accuracy: 0.1713 - val_loss: 3.7301 - val_accuracy: 0.1781
Epoch 7/40
3125/3125 [==============================] - 82s 26ms/step - loss: 3.5663 - accuracy: 0.1919 - val_loss: 3.4614 - val_accura

In [14]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

In [15]:
inputs = keras.Input(shape=(64,64,3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32,64,128,256,512]:
    residual = x
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(200, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [16]:
model.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [17]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="tiny_imagenet_feature_extraction_with_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [18]:
history = model.fit(
ds_train,
epochs=40,
batch_size = 32,
validation_data=ds_val,
callbacks=callbacks)

Epoch 1/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.9569 - accuracy: 0.0296 - val_loss: 4.6860 - val_accuracy: 0.0517
Epoch 2/40
3125/3125 [==============================] - 102s 33ms/step - loss: 4.6017 - accuracy: 0.0596 - val_loss: 4.5876 - val_accuracy: 0.0652
Epoch 3/40
3125/3125 [==============================] - 102s 33ms/step - loss: 4.4018 - accuracy: 0.0816 - val_loss: 4.4666 - val_accuracy: 0.0808
Epoch 4/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.2397 - accuracy: 0.1018 - val_loss: 4.2962 - val_accuracy: 0.1023
Epoch 5/40
3125/3125 [==============================] - 103s 33ms/step - loss: 4.1074 - accuracy: 0.1210 - val_loss: 4.2338 - val_accuracy: 0.1086
Epoch 6/40
3125/3125 [==============================] - 100s 32ms/step - loss: 3.9953 - accuracy: 0.1359 - val_loss: 4.0009 - val_accuracy: 0.1347
Epoch 7/40
3125/3125 [==============================] - 98s 31ms/step - loss: 3.9024 - accuracy: 0.1505 - val_loss: 3.

In [19]:
model = keras.models.load_model("tiny_imagenet_feature_extraction_with_data_augmentation.keras")

: 

: 